In [1]:
import json

from collections import defaultdict

In [4]:
with open("../DataProcessing/TestData/gameData.json", "r", encoding="utf-8") as file:
    data = json.load(file)

championCountsDict = defaultdict(lambda: defaultdict(int))
namesDict = defaultdict(list)

participantRoles = {
    1: 'Top', 6: 'Top',
    2: 'Jungle', 7: 'Jungle',
    3: 'Middle', 8: 'Middle',
    4: 'Bottom', 9: 'Bottom',
    5: 'Utility', 10: 'Utility'
}

mostMentionedNamesDict = {}
participantData = []

%store -r targetPuuid

targetPuuid = '9EG7Z2PyBlVS2VFnwfwMb1lpKF5WF5KxCwmqW2NJo9bI2750oFtrHFFDUW0lKAh46e1G7CXmJwv2AA'

In [5]:
# Function to calculate champion counts
def calculateChampionCounts(data):
    championCountsDict = {}
    namesDict = {}

    if 'info' in data and 'frames' in data['info']:
        for frame in data['info']['frames']:
            for event in frame.get('events', []):
                if event['type'] == 'CHAMPION_KILL':
                    killerId = int(event.get('killerId'))
                    victimId = int(event.get('victimId'))

                    if killerId not in championCountsDict:
                        championCountsDict[killerId] = {}
                        namesDict[killerId] = []

                    if victimId not in championCountsDict:
                        championCountsDict[victimId] = {}
                        namesDict[victimId] = []

                    # Find the killer's and victim's participant data
                    killerNames = [victim.get('name') for victim in event.get('victimDamageReceived', [])]
                    victimNames = [killer.get('name') for killer in event.get('victimDamageDealt', [])]

                    # Use the last name mentioned as the killer name
                    if killerNames:
                        killerName = killerNames[-1]
                        if killerName not in championCountsDict[killerId]:
                            championCountsDict[killerId][killerName] = 1
                        else:
                            championCountsDict[killerId][killerName] += 1
                        namesDict[killerId].append(killerName)

                    # Use the last name mentioned as the victim name
                    if victimNames:
                        victimName = victimNames[-1]
                        if victimName not in championCountsDict[victimId]:
                            championCountsDict[victimId][victimName] = 1
                        else:
                            championCountsDict[victimId][victimName] += 1
                        namesDict[victimId].append(victimName)
    return championCountsDict, namesDict

def calculateMostMentionedChampionNames(data):
    # Calculate champion counts
    championCountsDict, namesDict = calculateChampionCounts(data)

    # Process the counts and choose the most mentioned champion name for each ID
    mostMentionedNamesDict = {}
    for participantId, championCounts in championCountsDict.items():
        mostMentionedChampion = max(championCounts, key=championCounts.get)
        mostMentionedNamesDict[participantId] = mostMentionedChampion
        count = championCounts[mostMentionedChampion]
        names = namesDict[participantId]
    return mostMentionedNamesDict

mostMentionedNamesDict = calculateMostMentionedChampionNames(data)

def addParticipantData(participantId, puuid, championName, teamPosition):
    participantData.append([participantId, puuid, championName, teamPosition])

def findParticipantData(data):
    if 'info' in data and 'participants' in data['info']:
        for participant in data['info']['participants']:
            participantId = participant['participantId']
            puuid = participant['puuid']
            championName = mostMentionedNamesDict.get(participantId, "Unknown")
            teamPosition = participantRoles.get(participantId, "Unknown")  # Fetching team position
            
            addParticipantData(participantId, puuid, championName, teamPosition)

findParticipantData(data)

# Modified to include teamPosition
def findParticipantByPuuid(participantData, targetPuuid):
    for entry in participantData:
        participantId, puuid, championName, teamPosition = entry  # Unpacking with teamPosition
        if puuid == targetPuuid:
            return participantId, championName, teamPosition
    return None, None, None

# Run after populating participantData with the updated structure
targetParticipantId, targetChampionName, targetTeamPosition = findParticipantByPuuid(participantData, targetPuuid)

if targetParticipantId is not None:
    print(f"Participant ID: {targetParticipantId}, Champion: {targetChampionName}, Position: {targetTeamPosition}")
else:
    print("Participant not found.")

Participant not found.
